created on: Wed Jan 08 10:05:38 2020
Group 7
@author: L.D., C.T.D., C.L.

# Optimization of hyperparameters using Talos


This notebook requiers the installation of talos.


It aims to present a method to find the best hyperparameters in deep learning, equivalent to a gridSearch in Machine Learning.
The output is a CSV file summarizing for each combination :  
round_epochs,val_loss,val_accuracy,loss,accuracy
The last columns indicate the hyperparameters values(in this notebook, activation and conv_dropout).
The resulting CSV can be found in the same folder as this notebook.

By lack of time, this method has not been used on the models at the end of the project. 
However, with more time, it could be interesting to increase the accuracy of the models.

In [2]:
! pip install talos

     |████████████████████████████████| 26.6MB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 839kB 14.7MB/s eta 0:00:01
  Created wheel for talos: filename=talos-0.6.3-cp37-none-any.whl size=49626 sha256=139a352f29b98f9e0d4ebb53d72ff2798515b1262a16d3d443c4489a46c25452
  Stored in directory: /home/sid2019-18/.cache/pip/wheels/bb/d7/6b/86fd8b1fc7cfbd2c54796412f86efb5fb6a3a5c734014f6a66
  Created wheel for wrangle: filename=wrangle-0.6.7-cp37-none-any.whl size=49895 sha256=43af8090268828324f0888ff88f63c315058ea0a6756f0ff0c82b561ef9a3230
  Stored in directory: /home/sid2019-18/.cache/pip/wheels/bf/1b/50/d0403ce6ef269e364894da7b50db68db14c4ac62c577561e2d
  Created wheel for astetik: filename=astetik-1.9.9-cp37-none-any.whl size=56960 sha256=b98140ec7c62f6bc51313fc2bfe33e44950ec3f4e46a1e128aff981da8a0a384
  Stored in directory: /home/sid2019-18/.cache/pip/wheels/ae/70/21/c475cd079ec401dd6e1b9b1d42b4c38554ce12679bfb214aad
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3

In [9]:
#!/usr/bin/env python
# coding: utf-8

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from keras.models import load_model

# keras : librairie de deep learning
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, Dense, Flatten, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

import talos as ta

In [10]:
project_path = './../'
scrap_path = project_path + 'Scraping/'
airliners_path = scrap_path + 'Airliners/data/'

In [34]:
def reading(path = None, shape = (500,500), proba_train = 0.6, proba_test = 0.7):
    data_train = []
    data_test = []
    for i in tqdm(os.listdir(path)):
        if i[-3:] == 'jpg':
            rand = np.random.random()
            if rand <= proba_train:
                img = Image.open(path + i)
                data_train.append(np.mean(np.array(img.resize(shape, Image.BILINEAR)), axis=2))
            
            elif rand >= proba_test:
                img = Image.open(path + i)
                data_test.append(np.mean(np.array(img.resize(shape, Image.BILINEAR)), axis=2))
            
    return np.array(data_train), np.array(data_test)



In [35]:
A320_train, A320_test = reading(airliners_path + 'Airbus/A320/', (256,256))

In [36]:
A320_train.shape

(617, 256, 256)

In [37]:
A320_test.shape

(269, 256, 256)

In [38]:
A330_train, A330_test = reading(airliners_path + 'Airbus/A330/', (256,256))
A350_train, A350_test = reading(airliners_path + "Airbus/A350/", (256,256))
A380_train, A380_test = reading(airliners_path + "Airbus/A380/", (256,256))
Boeing737_train, Boeing737_test = reading(airliners_path + "Boeing/737/", (256,256))
Boeing747_train, Boeing747_test = reading(airliners_path + "Boeing/747/", (256,256))
Boeing767_train, Boeing767_test = reading(airliners_path + "Boeing/767/", (256,256))
Boeing777_train, Boeing777_test = reading(airliners_path + "Boeing/777/", (256,256))


In [39]:
yBoeing_train  = np.repeat('Boeing', len(Boeing777_train)+len(Boeing767_train)+ len(Boeing747_train) + len(Boeing737_train))
yAirbus_train = np.repeat('Airbus', len(A380_train) + len(A350_train) + len(A330_train)+ len(A320_train))

yBoeing_test  = np.repeat('Boeing', len(Boeing777_test)+len(Boeing767_test)+ len(Boeing747_test) + len(Boeing737_test))
yAirbus_test = np.repeat('Airbus', len(A380_test) + len(A350_test) + len(A330_test)+ len(A320_test))

In [45]:
ytrain = np.concatenate((yAirbus_train, yBoeing_train))
ytrain = pd.get_dummies(pd.Series(ytrain)).values

ytest = np.concatenate((yAirbus_test, yBoeing_test))
ytest = pd.get_dummies(pd.Series(ytest)).values

data_train = np.concatenate((A320_train,A330_train,A350_train,A380_train,
                             Boeing777_train,Boeing767_train,Boeing747_train,Boeing737_train))

data_test = np.concatenate((A320_test,A330_test,A350_test, A380_test,
                            Boeing777_test,Boeing767_test,Boeing747_test,Boeing737_test))

data_train = np.reshape(data_train, (data_train.shape[0], 256, 256, 1))

data_test = np.reshape(data_test, (data_test.shape[0], 256, 256, 1))

In [41]:
ytrain.shape

(4815, 2)

In [42]:
ytest.shape

(2378, 2)

In [46]:
data_train.shape

(4815, 256, 256, 1)

In [47]:
arr = np.arange(ytrain.shape[0])
np.random.shuffle(arr)

ytrain = ytrain[arr]
data_train = data_train[arr]


In [48]:
arr = np.arange(ytest.shape[0])
np.random.shuffle(arr)

ytest = ytest[arr]
data_test = data_test[arr]


In [59]:
def create_model(data_train, ytrain, data_test, ytest,  para, input_shape = (256,256,1), nb_couches = 5,
                 nb_neur = 4, kernel = (3, 3), pool = (2, 2),  nb_classes = ytrain.shape[1]):
    
    conv_dropout = para['conv_dropout'] 
     
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    for i in range(nb_couches):
        model.add(Conv2D(2**(nb_neur + i), kernel_size=kernel, activation = para['activation']))
        model.add(Dropout(conv_dropout))
     
        model.add(Conv2D(2**(nb_neur + i), kernel_size=kernel, activation = para['activation']))
        model.add(Dropout(conv_dropout))
       
        model.add(MaxPooling2D(pool_size = pool))

    model.add(Flatten())

    model.add(Dense(nb_classes, activation = 'softmax'))

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)
    
    out = model.fit(
        data_train, ytrain, epochs=10, batch_size=32, 
        verbose=0,
        validation_data=[data_test, ytest])
   
    return out, model


In [60]:
para = {'activation': ['relu', 'softmax'],
        'conv_dropout': [0.1, 0.2]
}
scan_results = ta.Scan(data_train, ytrain, para, create_model, 'talos_V0', x_val = data_test, y_val = ytest)



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

 25%|████████████████████                                                            | 1/4 [50:17<2:30:52, 3017.56s/it]

 50%|███████████████████████████████████████                                       | 2/4 [2:40:32<2:16:33, 4096.64s/it]

 75%|█████████████████████████████████████████████████████████                   | 3/4 [15:06:15<4:31:30, 16290.73s/it]

100%|██████████████████████████████████████████████████████████████████████████████| 4/4 [17:06:24<00:00, 15396.03s/it]
